In [221]:
from __future__ import print_function
from __future__ import unicode_literals
import yaml
import procesamiento_microdatos as pm
import pandas as pd
import unicodecsv as csv
import openpyxl as pyxl
from collections import OrderedDict

In [222]:
panel = pm.generate_panel()

In [223]:
with open("grupos.yaml") as groups_file:
    grupos = yaml.load(groups_file)
    
with open("alias.yaml") as alias_file:
    alias = yaml.load(alias_file)

In [229]:
def flatten(tree, flat_list=[]):
    """Toma una jerarquía de nodos (un 'arbol') y devuelve una lista plana con todos los nodos.
    
    Un arbol es una lista, que en cada posición tiene:
        - una string (un nodo 'base', sin hijos), o
        - un diccionario de una única clave (un nodo con hijos), donde
            - la clave es el nombre del nodo, y
            - el valor es una lista con los nodos hijos.
    """
    for node in tree:
        if isinstance(node, (unicode, str)):
            flat_list.append(node)
        else:
            for key in node:
                flat_list.append(key)
                flatten(node[key], flat_list)
    return flat_list


def unalias(df, alias=alias):
    """Usa el dict 'alias' para dar nombres completos al índice y las columnas de un DataFrame"""
    return df.rename(columns=alias, index=alias)


def get_yr(yr):
    """Devuelve la data correspondiente a un año del panel de microdatos con los nombres de energías y usos completos."""
    data = unalias(panel[yr])
    data = corregir_signo_consumo(data)
    return data

In [230]:
def corregir_signo_consumo(df):
    # Corrijo signo de rubros de consumo para que "reciban" de las distintas formas de energía
    for consumo in ["Consumo No Energético", "Residencial", "Comercial", "Transporte", "Agropecuario", "Industria"]:
        df[consumo] = -df[consumo]
    return df

In [239]:
def make_links(yr):
    data = get_yr(yr)
    links = list()
    for row in data.index.get_values():
        for col in data.columns.get_values():
            value = data.loc[row, col]
            if value < 0:
                links.append({"source": row, "target": col, "value": abs(value)})
            elif value > 0:
                links.append({"source": col, "target": row, "value": abs(value)})
    return links

In [240]:
data1992 = get_yr(1992)

In [243]:
links2010 = make_links(2010)

In [244]:
len(links2010)

188

In [193]:
def get_nodos(formato="dict"):
    wb = pyxl.load_workbook("maestro-nodos.xlsx")
    ws = wb.active
    raw_data = ws.values
    
    cols = next(raw_data)
    rows = list(raw_data)
    if formato == "dict":
        return [dict(zip(cols, row)) for row in rows]
    elif formato == "df":
        return pd.DataFrame.from_records(rows, columns=cols)
    else:
        print("Formato no reconodico: {}".format(formato))

In [194]:
nodos = get_nodos("df")

In [155]:
data_energias = data2010.reset_index().rename_axis({"index": "Nombre"}, 1)

In [156]:
data_energias = nodos.merge(data_energias)

In [176]:
agrupamientos_usos = [
    ("Otros Conceptos de Oferta", ['Variación de Stock', 'Búnker', 'No Aprovechado', 'Ajustes']),
    ("Oferta", ["Producción", "Importación", "Exportación", "Pérdidas", "Otros Conceptos de Oferta"]),
    ("Centrales Eléctricas", ['Servicio Público', 'Autoproducción']),
    ("Otros Centros de Transformación", ['Aceiteras y Destilerías', 'Coquerías', 'Carboneras', 'Altos Hornos']),
    ("Centros de Transformación", ["Centrales Eléctricas", "Plantas de Tratamiento de Gas", "Refinerías", "Otros Centros de Transformación"]),
    ("Consumo Final", ["Consumo No Energético", "Residencial", "Transporte", "Comercial", "Industria", "Agropecuario"]),
    ("Consumo", ["Consumo Propio", "Consumo Final"])
]


In [177]:
for (padre, hijos) in agrupamientos_usos:
    print(padre)
    data_energias.loc[:, padre] = reduce(pd.Series.add, [data_energias.loc[:, hijo] for hijo in hijos])

Otros Conceptos de Oferta
Oferta
Centrales Eléctricas
Otros Centros de Transformación
Centros de Transformación
Consumo Final
Consumo


In [180]:
data_energias.loc[:, ["Oferta", "Otros Conceptos de Oferta", 'Variación de Stock', 'Búnker', 'No Aprovechado', 'Ajustes', "Producción", "Importación", "Exportación", "Pérdidas"]]

,Oferta,Otros Conceptos de Oferta,Variación de Stock,Búnker,No Aprovechado,Ajustes,Producción,Importación,Exportación,Pérdidas
0,3644.025125,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,3680.465376,0.000000,0.000000,-36.440251
1,2282.653516,-3.464843e-01,0.000000,0.000000,0.000000,-3.464843e-01,0.000000,2283.000000,0.000000,0.000000
2,41255.012199,-1.186566e+03,0.000000,0.000000,-1148.916014,-3.764968e+01,41501.000000,1826.830000,-132.951240,-753.300872
3,26460.427302,7.505960e+02,172.973916,0.000000,0.000000,5.776221e+02,30437.321712,0.000000,-4727.490387,0.000000
4,131.108875,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,131.108875,0.000000,0.000000,0.000000
5,0.007387,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.007387,0.000000,0.000000,0.000000
6,1159.897911,-1.013715e+02,22.000000,0.000000,0.000000,-1.233715e+02,35.063700,1252.471330,-26.265637,0.000000
7,65.181317,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,65.181317,0.000000,0.000000,0.000000
8,1717.354837,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,1717.354837,0.000000,0.000000,0.000000
9,815.758020,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,815.758020,0.000000,0.000000,0.000000


In [89]:
data2010.loc["Energía Hidráulica"]

Producción                       3680.465376
Importación                         0.000000
Variación de Stock                  0.000000
Exportación                         0.000000
Búnker                              0.000000
No Aprovechado                      0.000000
Pérdidas                          -36.440251
Ajustes                             0.000000
Servicio Público                -3636.277277
Autoproducción                     -7.747847
Plantas de Tratamiento de Gas       0.000000
Refinerías                          0.000000
Aceiteras y Destilerías             0.000000
Coquerías                           0.000000
Carboneras                          0.000000
Altos Hornos                        0.000000
Consumo Propio                      0.000000
No Energético                       0.000000
Residencial                         0.000000
Comercial                           0.000000
Transporte                          0.000000
Agropecuario                        0.000000
Industria 

In [181]:
data2010

,Producción,Importación,Variación de Stock,Exportación,Búnker,No Aprovechado,Pérdidas,Ajustes,Servicio Público,Autoproducción,...,Coquerías,Carboneras,Altos Hornos,Consumo Propio,Consumo No Energético,Residencial,Comercial,Transporte,Agropecuario,Industria
Aceites Vegetales,1717.354837,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Alcohol Vegetal,65.181317,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Bagazo,850.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-115.295063,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,734.754937
Biodiesel,1647.286760,0.000000,0.000000,-1194.663020,0.000000,0.000000,0.000000,6.050715e-15,-0.429870,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Bioetanol,63.284541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.608733e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Carbón Mineral,35.063700,1252.471330,22.000000,-26.265637,0.000000,0.000000,0.000000,-1.233715e+02,-424.713780,-14.252040,...,-710.337856,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.594235
Carbón Residual,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Carbón Vegetal,262.826900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,262.826900,0.000000,0.000000,0.000000,157.696140,105.130760,0.000000,0.000000,0.000000
Coque de Carbón,532.753392,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,532.753392,0.000000,-532.753392,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Coque de Petróleo,950.676033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.239668e-01,0.000000,0.000000,...,0.000000,0.000000,-637.000000,0.000000,314.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [186]:
for row in data2010.index.get_values():
    for col in data2010.columns.get_values():
        value = data2010.loc[row, col]
        if value < 0:
            print("De {} a {} van {}".format(row, col, abs(value)))
        elif value > 0:
            print("De {} a {} van {}".format(col, row, abs(value)))

De Aceites Vegetales a Producción van 1717.35483736
De Aceiteras y Destilerías a Aceites Vegetales van 1717.35483736
De Alcohol Vegetal a Producción van 65.1813171284
De Aceiteras y Destilerías a Alcohol Vegetal van 65.1813171284
De Bagazo a Producción van 850.05
De Autoproducción a Bagazo van 115.2950625
De Bagazo a Industria van 734.7549375
De Biodiesel a Producción van 1647.28676
De Exportación a Biodiesel van 1194.66302
De Biodiesel a Ajustes van 6.05071548421e-15
De Servicio Público a Biodiesel van 0.42987
De Refinerías a Biodiesel van 452.19387
De Biodiesel a Aceiteras y Destilerías van 1647.28676
De Bioetanol a Producción van 63.2845408
De Ajustes a Bioetanol van 3.60873344
De Refinerías a Bioetanol van 59.67580736
De Bioetanol a Aceiteras y Destilerías van 63.2845408
De Carbón Mineral a Producción van 35.0637
De Carbón Mineral a Importación van 1252.47132961
De Carbón Mineral a Variación de Stock van 22.0
De Exportación a Carbón Mineral van 26.2656368
De Ajustes a Carbón Minera